In [1]:
from torch.utils.data import DataLoader 
import torch
from pyscf import gto, dft, scf 

from dpyscfl.scf import SCF
from dpyscfl.net import XC, X_L, C_L
from dpyscfl.utils import *

from ase import Atoms
from ase.io import read

In [2]:
traj = read('/home/awills/Documents/Research/xcdiff/data/haunschild_pbe.traj', ':')

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ase/io/jsonio.py:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(obj)


In [14]:
[(i.get_chemical_formula(), i.info) for i in traj]
mols = [gto.M(atom=[[ispec,ipos] for ispec,ipos in zip(iat.get_chemical_formula(), iat.positions)], spin=None, charge=0) for iat in traj[:2]]

In [17]:
mols[0].spin

1

### 1. Define the xc model

Here we use a simple LDA model (`level = 1`), by choosing `heg_mult = True` (for homogeneous electron gas exchange) and `pw_mult = True` (for Perdew Wang HEG correlation), without any enhancement factors (`grid_models = []`). No Hartree-Fock exact exchange is used `exx_a=None`.

In [32]:
c = C_L(n_input=3) #3 for gga
x = X_L(n_input=3) #3 for GGA

xc = XC(grid_models = [c], heg_mult=False, pw_mult=False, level=2, exx_a=1)
#xc = XC(grid_models = [x], heg_mult=False, pw_mult=False, level=2, exx_a=None)

#xc = XC(grid_models = [x,c], heg_mult=False, pw_mult=False, level=2, exx_a=None)

#xc = XC(grid_models = [], heg_mult=True, pw_mult=True, level=1, exx_a=None)

xc.epsilon = 1e-5

Create the object that runs the SCF calculation

In [49]:
alpha_init = 0.8
alpha = torch.nn.Parameter(torch.Tensor([alpha_init]))
dscf = SCF(alpha=alpha, nsteps=10, xc=xc, device='cpu', exx=False)

### 2. Create a molecule with pyscf

This computes all overlap integrals and other information that will be used by our differentiable implementation

In [22]:
mol = gto.M(atom='H 0 0 0; H 0 0 1', basis='3-21G')
mf = scf.RHF(mol)
mf.grids.level = 1
mf.grids.build()
mf.kernel()
E_converged = mf.e_tot

AttributeError: 'RHF' object has no attribute 'grids'

In [23]:
mf.xc

AttributeError: 'RHF' object has no attribute 'xc'

### 3. Create input data structures

In [51]:
# * The marked lines are just workarounds because we are only dealing with one system in this tutorial and both Dataset and DataLoader are designed 
# to deal with multiple data points 

matrices = get_datapoint(mol, mf)
matrices = {key: [val] for key,val in matrices.items()} # *
dset = Dataset(**matrices)
dloader = DataLoader(dset, batch_size = 1)
inputs = next(iter(dloader)) # *

In [61]:
inputs

[tensor([[[ 0.2044,  0.2861, -0.0113,  0.0025],
          [ 0.2861,  0.4022,  0.0025,  0.0291],
          [-0.0113,  0.0025,  0.2044,  0.2861],
          [ 0.0025,  0.0291,  0.2861,  0.4022]]]),
 {'v': tensor([[[-2.2498, -1.1221, -0.4011, -0.6727],
           [-1.1221, -1.1562, -0.6727, -0.8873],
           [-0.4011, -0.6727, -2.2498, -1.1221],
           [-0.6727, -0.8873, -1.1221, -1.1562]]]),
  't': tensor([[[ 1.5494,  0.2932, -0.0142,  0.1092],
           [ 0.2932,  0.2748,  0.1092,  0.1549],
           [-0.0142,  0.1092,  1.5494,  0.2932],
           [ 0.1092,  0.1549,  0.2932,  0.2748]]]),
  's': tensor([[[1.0000, 0.6459, 0.1999, 0.3764],
           [0.6459, 1.0000, 0.3764, 0.7210],
           [0.1999, 0.3764, 1.0000, 0.6459],
           [0.3764, 0.7210, 0.6459, 1.0000]]]),
  'n_elec': tensor([2]),
  'n_atoms': tensor([2]),
  'e_nuc': tensor([0.5292]),
  'mo_energy': tensor([[-0.3317,  0.0163,  0.8137,  0.9950]]),
  'mo_occ': tensor([[2., 0., 0., 0.]]),
  'eri': tensor([[[[[1.140

### 4. Run the calculation

In [52]:
output = dscf.forward(*inputs)

In [37]:
output

{'E': tensor([-0.1824, -0.4248, -0.4077, -0.4072, -0.4071, -0.4071, -0.4071, -0.4071,
         -0.4071, -0.4071], grad_fn=<CatBackward0>),
 'dm': tensor([[0.0608, 0.1484, 0.0608, 0.1484],
         [0.1484, 0.3623, 0.1484, 0.3623],
         [0.0608, 0.1484, 0.0608, 0.1484],
         [0.1484, 0.3623, 0.1484, 0.3623]], grad_fn=<PermuteBackward0>),
 'mo_energy': tensor([0.0613, 0.3411, 1.1934, 1.4133], grad_fn=<SymeigBackward0>)}

## 5. Calculate the loss

In [38]:
E_converged_t = torch.Tensor([E_converged])

E_converged_t

tensor([-1.1075])

Mean squared error loss:

In [39]:
torch.sum((output['E'] - E_converged_t)**2)

tensor(5.2453, grad_fn=<SumBackward0>)

PyTorch-native MSE loss 

In [40]:
loss = torch.nn.MSELoss()
loss_value = loss(output['E'], E_converged_t.repeat(10))
loss_value

tensor(0.5245, grad_fn=<MseLossBackward0>)

Calculate gradients with backprop

In [41]:
loss_value.backward()

In [42]:
alpha.grad

tensor([-0.0233])

Optimizer

In [43]:
opt = torch.optim.SGD([alpha], lr=0.001)
opt.step()

In [44]:
print('Old alpha:', alpha_init)
print('New alpha:', alpha.detach().numpy()[0])

Old alpha: 0.8
New alpha: 0.8000232706344446


In [58]:
loss = torch.nn.MSELoss()
opt = torch.optim.SGD([alpha], lr=0.001)

for i in range(50):
    print(i)
    ioutput = dscf.forward(*inputs)
    loss_value = loss(ioutput['E'], E_converged_t.repeat(10))
    loss_value.backward()
    opt.step()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [59]:
print(alpha_init, alpha)

0.8 Parameter containing:
tensor([0.8672], requires_grad=True)


In [60]:
loss_value

tensor(0.5233, grad_fn=<MseLossBackward0>)